## Основы pytorch

Для выполнения ДЗ создайте приватный репозиторий и добавьте `https://github.com/norsage` в collaborators (Settings -> Collaborators -> Add people)

In [3]:
pip install torch

<class 'OSError'>: Not available

In [1]:
import torch

<class 'ModuleNotFoundError'>: No module named 'torch'

#### 1. Операции над тензорами (1 балл)

##### 1.1 Среднее значение по столбцам

In [2]:
torch.manual_seed(42)
x = torch.randint(10, size=(2, 3)).float()
x

tensor([[2., 7., 6.],
        [4., 6., 5.]])

In [3]:
mean_by_row = x.mean(dim=0)
assert torch.allclose(
    mean_by_row, _expected := torch.tensor([3.0, 6.5, 5.5])
), f"{mean_by_row} != {_expected}"

##### 1.2. Взвешенное среднее
В тензоре `w` находятся ненормализованные веса для расчёта взвешенных средних тензора `x` по строкам.

Найдите эти взвешенные средние, получая нормализованные веса с помощью функции `torch.softmax` (или метода `.softmax`)

In [4]:
torch.manual_seed(42)
x = torch.randint(10, size=(2, 3)).float()
w = torch.randint(10, size=(2, 3)).float()
print(x)
print(w)

tensor([[2., 7., 6.],
        [4., 6., 5.]])
tensor([[0., 4., 0.],
        [3., 8., 4.]])


In [5]:
w_avg = (w.softmax(dim=1) * x).sum(dim=1)
assert torch.allclose(
    w_avg, _expected := torch.tensor([6.8940, 5.9690])
), f"{w_avg} != {_expected}"

##### 1.3. Умножение матриц на векторы

В тензоре `m` - две матрицы, нужно сделать тензор, в котором i-й элемент - результат умножения матрицы `m[i]` на вектор `x[i]`.

Это можно было бы сделать так: `torch.stack([m[i] @ x[i] for i in len(m)])`.

Попробуйте найти решение без цикла.

In [6]:
torch.manual_seed(42)
x = torch.randint(10, size=(2, 3)).float()
m = torch.randint(10, size=(2, 3, 3)).float()
print(m)
print(x)

tensor([[[0., 4., 0.],
         [3., 8., 4.],
         [0., 4., 1.]],

        [[2., 5., 5.],
         [7., 6., 9.],
         [6., 3., 1.]]])
tensor([[2., 7., 6.],
        [4., 6., 5.]])


In [7]:
matmul = torch.diagonal(torch.transpose(m @ x.T, 1, 2), dim1=0, dim2=1).T
assert torch.allclose(
    matmul, _expected := torch.tensor([[28.0, 86.0, 34.0], [63.0, 109.0, 47.0]])
), f"{matmul} != {_expected}"

##### 1.4. Матрица попарных расстояний

Даны две матрицы `x` и `y`, нужно получить матрицу `d`, где `d[i, j]` - евклидово расстояние между векторами `x[i]` и `y[j]`.

Подсказка 1: воспользуйтесь broadcasting и добавлением размерностей в исходные тензоры.

Подсказка 2: можно не считать евклидово расстояние вручную, есть функция `torch.linalg.norm`



In [8]:
torch.manual_seed(42)
x = torch.randint(10, size=(2, 3)).float()
y = torch.randint(10, size=(3, 3)).float()
print(x)
print(y)

tensor([[2., 7., 6.],
        [4., 6., 5.]])
tensor([[0., 4., 0.],
        [3., 8., 4.],
        [0., 4., 1.]])


In [9]:
pdist = torch.linalg.norm(x.unsqueeze(1) - y, dim=2)
assert torch.allclose(
    pdist,
    _expected := torch.tensor([[7.0000, 2.4495, 6.1644], [6.7082, 2.4495, 6.0000]]),
), f"{pdist} != {_expected}"

#### 2. Функция Power (1 балл)
Используя сложение и умножение, реализуйте возведение в целочисленную степень FloatTensor как функцию autograd (т.е. наследника `torch.autograd.Function`)

In [10]:
class Power(torch.autograd.Function):
    @staticmethod
    def forward(tensor, p):
        return tensor**p

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx is a context object that can be used to stash information
        # for backward computation
        tensor, p = inputs
        ctx.save_for_backward(tensor)
        ctx.p = p

    @staticmethod
    def backward(ctx, grad_output):
        # We return as many input gradients as there were arguments.
        # Gradients of non-Tensor arguments to forward must be None.
        tensor, = ctx.saved_tensors
        p = ctx.p
        return grad_output * p * tensor**(p-1), None

In [11]:
assert torch.all(Power.apply(torch.tensor([1, 2, 3]), 0) == torch.tensor([1, 1, 1]))
assert torch.all(Power.apply(torch.tensor([1, 2, 3]), 2) == torch.tensor([1, 4, 9]))

TypeError: all() received an invalid combination of arguments - got (bool), but expected one of:
 * (Tensor input, *, Tensor out = None)
 * (Tensor input, tuple of ints dim = None, bool keepdim = False, *, Tensor out = None)
 * (Tensor input, int dim, bool keepdim = False, *, Tensor out = None)
 * (Tensor input, name dim, bool keepdim = False, *, Tensor out = None)


#### 3. Многочлен (3 балла)
Найдите корень (он один) заданного полинома (очень хорошего!) с точностью до пяти знаков после запятой:
1. Используя бинарный поиск https://en.wikipedia.org/wiki/Binary_search_algorithm
2. Используя метод Ньютона https://en.wikipedia.org/wiki/Newton%27s_method
   
   Задаётся начальное приближение вблизи предположительного корня, после чего строится касательная к графику исследуемой функции в точке приближения, для которой находится пересечение с осью абсцисс. Эта точка берётся в качестве следующего приближения. И так далее, пока не будет достигнута необходимая точность.
   
   (hint: для вычисления производных используйте метод `backward()`)
   
   $x_{n+1} = x_{n} - \frac{f(x_n)}{f'(x_n)}$

Сравните скорость методов с помощью `%%timeit`, т.е. оцените, какой из них найдёт ответ быстрее

In [ ]:
from typing import Callable

Polynomial = Callable[[torch.FloatTensor], torch.FloatTensor]
torch.set_printoptions(precision=6)

def poly(x: torch.FloatTensor) -> torch.FloatTensor:
    return x**7 + 5 * x**3 + 17 * x - 9

In [ ]:
def bin_search_find_zero(poly: Polynomial) -> torch.FloatTensor:
    """Функция для бинарного поиска"""
    l = torch.FloatTensor([0.])
    r = torch.FloatTensor([1.])
    tol = 1e-5
    while r - l > torch.FloatTensor([tol]):
        middle = l + (r - l) / 2
        if poly(middle) < 0:
            l = middle
        else:
            r = middle
    return l

In [ ]:
def newton_find_zero(poly: Polynomial) -> torch.FloatTensor:
    """Функция для метода Ньютона"""

    # первое приближение (не забываем про то, что понадобится градиент!)
    x = torch.tensor([0.], dtype=torch.float, requires_grad=True)

    # останавливаемся, если значение функции достаточно близко к нулю
    tol = 10**-5

    # значение
    val = poly(x)

    # цикл обновления
    while abs(val) > tol:  # когда останавливаемся?
        # получаем градиент, обновляем значение x, оцениваем f(x)
        # hint: нужны ли нам градиенты, когда мы обновляем x?
        # hint: не забываем про обнуление градиента с прошлых шагов
        val.backward()
        with torch.no_grad():
            x = x - val / x.grad
        x.requires_grad = True
        val = poly(x)
        x.grad = None
    return x

In [ ]:
x = newton_find_zero(poly)
print(x)
print(poly(x))

In [ ]:
%%timeit
x = bin_search_find_zero(poly)

In [ ]:
%%timeit
x = newton_find_zero(poly)